# Single-neuron models

Neuroscience is studied at a variety of spatial and functional scales: from molecules through single neurons to networks and systems. Decades of combined electrophysiology and theory work at the _cellular level_ have nicely exemplified the type of reductionist work that is driving our understand of neural systems forward. It is about recapitulating complex biological phenomena in a couple of equations, which then form the basis for further studies at a larger scale (e.g. networks). Thus, we will start our journey in computational neuroscience by taking some time to understand what neurons are, how they respond to messages sent by other neurons, and how they generate their own messages.

This notebook is divided into two tutorial parts, followed by your first piece of coursework. The first part concerns the famous Hodgkin-Huxley model, covered in lectures. The model is fully implemented in module `Pkp.Neuron.HH`, and you use the Jupyter notebook to explore its behaviour, including the effect of different parameters on the input/output transfer function. The second part concerns reduced models of the “integrate-and-fire” type, which are no longer based on exact biophysics yet capture the response properties of real neurons very accurately. Another objective of this notebook is to progressively ease you into writing more and more complex OCaml code.

In [ ]:
;;
#require "pkp"

The following prevents Owl from printing the contents of matrices in the cell output; this is especially useful here as we will deal with fairly large matrices. If you want to revert that, do this instead:
```ocaml
let () = Pkp.Misc.hooting_owl ()
```

In [ ]:
let () = Pkp.Misc.quiet_owl ()

Let's open the `Pkp.Neuron` module once and for all since we're going to use it a lot. I find it convenient to make an alias `A`, this is going to shorten most cell outputs (types etc):

In [ ]:
module A = Pkp.Neuron

open A

open Owl

open Gp

Take some time to look at the [documentation](https://pkp-neuro.github.io/pkp-tutorials/pkp/Pkp/Neuron/index.html) of the `Neuron.HH` module before moving on.

## 0. Preliminaries: differential equations

(Skip this if you are already familiar with those). Let's take a few minutes to understand differential equations, as they will come up quite often in this course. The ones we will encounter are of the type

$$ \frac{dx}{dt} =  f[x(t), u(t)] \qquad \text{with known initial condition } x(t=0)$$

where $x$ is some state variable, and $f(x,u)$ is some given function of the state $x(t)$ itself as well as some other (possibly time-varying) signal $u(t)$. In plain English, this equation says: 

> “At any time $t$, the slope of the $x(t)$ curve is given by $f[x(t), u(t)]$.”


That's it. If we already know $x$ at some time, say $t=0$, then the equation above is enough information to fully reconstruct the whole time series of $x$. Think about how you'd draw it: you'd take some small time steps of size $\delta_t$, and update $x$ following the local slope:

$$ x(t+\delta_t) = x(t) + \delta_t f[x(t),u(t)] $$

and move on to the next time step. It turns out this simple method is a tad too simplistic, as errors tend to accumulate unless $\delta_t$ is tiny, in which case integration is very slow. Below, we will use more efficient solvers provided by the `Owl_ode` library.

Let's go through a simple example, one that does not even involve input $u$:

$$ \frac{dx}{dt} = -\frac{x}\tau \quad \text{with} \quad x(t=0)=a $$

The exact solution to this simple “linear, first-order, ordinary differential equation” is in fact known: you can check for yourself that it is

$$x(t) = a\ \exp\left(-\frac{t}{\tau}\right) $$

which is plotted below:

In [ ]:
let a = 1.2
and tau = 0.2

let solution t = a *. exp (-.t /. tau)

(* define plot properties now, we'll reuse them later *)
let props = default_props @ [ xlabel "time t"; ylabel "solution x(t)" ]

let () =
  let fig (module P : Plot) = P.plot (F (solution, Mat.linspace 0. 1. 100)) props in
  Juplot.draw ~size:(300, 200) fig

Let's compare that to the output of our `Owl_ode` solver:

In [ ]:
let () =
  (* solve the differential equation *)
  let t, x =
    let open Owl_ode in
    let dxdt x _ = Mat.(neg x /$ tau) in
    let tspec = Types.(T1 { t0 = 0.0; duration = 1.0; dt = 1E-1 }) in
    let solver = Native.D.rk45 ~tol:1E-5 ~dtmax:0.1 in
    (* let solver = Native.D.euler in *)
    Ode.odeint solver dxdt (Mat.create 1 1 a) tspec ()
  in
  (* plot result *)
  let fig (module P : Plot) =
    P.plots
      [ item (F (solution, Mat.linspace 0. 1. 100)) ~style:"l lc 7 lw 2"
      ; item (L [ t; x ]) ~style:"p pt 7 lc 8 ps 0.3 lw 2"
      ]
      props
  in
  Juplot.draw ~size:(300, 200) fig

Can't see the red curve? That's a good sign.

<span style="color: #ffac00; font-weight:bold;">TODO:</span>
1. Play with parameter $\tau$: how does it affect the solution?
2. Add a constant input $u$ to the equation, so it becomes $ dx/dt = (-x+u)/\tau$. Set e.g. `u=5.0`. How does that affect the solution? Could you have predicted the steady-state value without simulating the sytem?

## 1. Hodgkin-Huxley model

Here is how to simulate a HH neuron for 100 ms with zero input:

In [ ]:
let time, state = HH.simulate ~prms:HH.default_prms ~duration:0.1 (fun _ -> 0.)

Here is a function that takes the result of a HH simulation (`Owl.Mat * Owl.Mat`, cf. documentation) and plots the voltage timecourse:

In [ ]:
let plot_voltage (time, state) =
  let figure (module P : Plot) =
    P.plot
      (L [ time; state ])
      ~using:"1:2"
      ~style:"l lc 8"
      (default_props
      @ [ xlabel "time (s)"
        ; set "yrange [*<-90:-55<*]" (* autoscaling within bounds *)
        ; ytics (`regular [ -100.; 20. ])
        ; ylabel "V_m (mV)"
        ])
  in
  Juplot.draw ~size:(600, 200) figure

<span style="color: #ffac00; font-weight:bold;">TODO:</span> Use this function to display the result of your simulation above.

In [ ]:
(* your code here *)

### 1.1 Simulation with constant input current

<span style="color: #ffac00; font-weight:bold;">TODO:</span> Now write code to define this input current (function of time):

$$ u(t) = \left\{ \begin{array}{ll}
  0 & \text{if } t<0.04\\
  h & \text{otherwise}
  \end{array}\right.
$$

where $h$ is some constant current. Re-run the `HH.simulate` code above, now with this input function in place of `(fun _ -> 0.)`. Adjust $h$, starting from some very low value (remember, we are talking about `< 1E-9` amperes), increasing it until you get an action potential. What is the threshold $h$ value, approximately? (you may want to play with the simulation `duration` as well).

In [ ]:
(* your code here *)

<span style="color: #ffa500; font-weight:bold;">ADVANCED:</span> Now, write a function `plot_all` similar to `plot_voltage` above, but which adds another plot (underneath the main voltage plot) showing the timecourse of the 3 gate variables. Use this function to inspect the behaviour of the gate variables ─ this should help you understand the model.

In [ ]:
(* your advanced code here ^^ *)

### 1.2 Anode break excitation

Simulate the HH model using the following _negative_ (hyperpolarising) input current:

$$ u(t) = \left\{ \begin{array}{ll}
   -h & \text{if } t<0.04\\
   0  & \text{otherwise}
   \end{array} \right. $$
   
where $h>0$. Try `h=0.4E-9` (0.4 nA). What happens? Is there a minimum value of `h` for which this phenomenon occurs?

## 2. Leaky integrate-and-fire (LIF) model

Redo all the steps above, now using the `LIF` module.

What can you infer about the key qualitative similarities and differences between the HH and LIF models?

## 3. Some solutions

☝ Don't look below ↓ unless you are really stuck! ☝

In [ ]:
let _ =
  let u t = if t < 0.04 then 0. else 0.64E-9 in
  let t, vm = HH.simulate ~prms:HH.default_prms ~duration:0.6 u in
  plot_voltage (t, vm)

In [ ]:
let plot_all (time, state) =
  let figure (module P : Plot) =
    (* membrane potential *)
    P.plot
      (L [ time; state ])
      ~using:"1:2"
      ~style:"l lc 8"
      (default_props
      @ [ margins [ `left 0.2; `right 0.9; `top 0.9; `bottom 0.55 ]
        ; borders [ `left ]
        ; set "yrange [*<-90:-55<*]" (* autoscaling within bounds *)
        ; ytics (`regular [ -100.; 20. ])
        ; ylabel "V_m (mV)"
        ; unset "xtics"
        ]);
    (* the 3 gate variables *)
    P.plots
      List.(
        init 3 (fun j ->
            let g = Mat.col state (j + 1) in
            item (L [ time; g ]) ~style:Printf.(sprintf "l lc %i" j)))
      (default_props
      @ [ margins [ `left 0.2; `right 0.9; `top 0.5; `bottom 0.2 ]
        ; xtics (`regular [ 0.; 0.05 ])
        ; xlabel "time"
        ; yrange (0., 1.01)
        ; ylabel "gate"
        ])
  in
  Juplot.draw ~size:(600, 400) figure

In [ ]:
let _ =
  let input t = if t < 0.02 then 0. else 0.34E-9 in
  LIF.simulate ~prms:LIF.default_prms ~duration:0.1 input
  |> plot_voltage 

---

# COURSEWORK 1

<span style="color: #ffac00; font-weight: bold;">SUBMISSION DEADLINE:</span> Thursday 8 August 6pm. 

Please answer both questions, in less than 500 words each (+ code).

- Write your answers directly in markdown cells in this notebook.
- Do include simulation code + figures, but not limit your answer to these; instead, write a _narrative_.
- Make sure to submit a clean notebook; in particular:
   - all cells must have been evaluated without errors; do not “clear outputs” before submitting
   - the whole notebook is *linear*, i.e. can be re-evaluated (e.g. by me!) by only moving forward cell by cell.
   
To submit, simply drag-and-drop your file in a direct message to me on [slack](https://pkp-neuro.slack.com).

**Please replace this line with your student PID**

## Question 1

Investigate the response of the HH and LIF models (with their default parameters) to some constant input `h`. Specifically, for each model:
1. Plot the steady-state firing rate (number of spikes per second, discarding any transient phase) as a function of `h`, over an “interesting range of `h`” (which you must determine) so we can clearly see the firing rate increasing from zero to tens of Hz as `h` increases.
2. Comment on the main qualitative differences between the two models.

Hint: you can use the function `Pkp.Neuron.count_spikes`.

# Question 2

Investigate the “frequency response” (see below) of the HH and LIF models (with their default parameters). Specifically, consider injecting a current of the form

$$ h(t) = a\ \cos (2\pi f t) $$ 

where $f$ is some stimulation frequency (in Hz), and `a=0.1E-9` is a constant small enough so that the input does not cause the neuron to fire an action potential.
Since the input is periodic, so will the output (after some transient).

To convince yourself that that's true, begin by looking at the voltage response for `f=20.0` Hz.

Next, plot the amplitude of the $V_m$ oscillations (peak-to-trough), as a function of $h$ (Hint: you might want to use `Mat.max'` and `Mat.min'`). Is there any difference between the HH and LIF models?